In [15]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time
from random import randint

In [16]:
options = webdriver.ChromeOptions() 
options.add_argument("download.default_directory=" + os.getcwd() + "/downloads")

In [17]:
start_page = "https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG="
test_name = "david bermudes"
print(start_page)

https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG=


In [18]:
#Use pandas create dataframe (df) object that will store the data
columns = ['name','paper_type','citation','url','abstract','title','collaborators']
citations = pd.DataFrame()
#Place the list of names generated previously in the project into the df object
citations

""


In [19]:
#df1['e'] = Series(np.random.randn(sLength), index=df1.index)
#citations['a'] = pd.Series(test_name, index=citations.index)


Process Flow
-------------------
Search professors from start_page by name
Click Search
Find the first reseach item in the research section
Determine type of publication
Get Abstract if Abstract exists, if not skip to next
Click citation download link
Select option (1 = RIS, 2 = BibTex, 3 = Plain text)
Select option  (1 = citation 2 = citation and abstract)
Download
Go to most recent downloaded file
Read text from that file
Place citation into variable
If abstract exists for citation place into abstract variable if no abstract found insert "No abstract found."

In [20]:
driver = webdriver.Chrome()
driver.implicitly_wait(10) # seconds loading implicitly

#Set driver to go to start_page and search a given name with valid scholar profile
driver.get(google_scholar)
print(driver.current_url)

https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG=


In [21]:
#def google_scholar(url):
    
google_scholar = "https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG="

#Instatiate the webdriver and set to implicitly wait for page objects to load

search_bar = driver.find_element_by_class_name('gs_in_txt')
search_bar.send_keys(test_name)
search_bar.send_keys(Keys.ENTER)


#Set driver to click the first profile given as a search result. Will need to be modified to be more general later.
view_profile_link = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[1]/div/div/h3/a')
view_profile_link.click()

# To get all the links we need to click the Show More multiple times to show the whole links 

gs_bpf_wrapper = driver.find_element_by_id("gsc_bpf_more")

while not gs_bpf_wrapper.get_attribute("disabled"):
    gs_bpf_wrapper.click()
    time.sleep(10)

print("Not in an infinite loop")

#pull the first citation for the page
view_citation_links = driver.find_elements_by_class_name('gsc_a_at')
print(view_citation_links)

for link in view_citation_links:
    link.click()
    title = link.text
    print(title)
    citation_table = driver.find_element_by_id("gsc_vcd_table")
    citation_table_data_objects= citation_table.find_element_by_class_name("gs_scl")
    
    

    
#view_citation_link.click()

#Citation information title, publication date, journal, volume, issue, pages, publisher, abstract.

citations = ['Professor','Authors','Publication_Title', 'URL','Citation']





citation = {
    "Authors" : "",
    "Title" : "",
    "Date" : "",
    "Publisher": "",
    "Description": "",
    "Journal": "",
    "Issue": "",
    "Volume":"",
    "Pages": ""
}

counter = 0

for data in citation_table_data_objects:
    
    data_category = data.find_element_by_tag_name("div")
    #print(data_category.text)
    
    
        
    if data_category.text == "Authors":
        data_field = data.find_element_by_class_name("gsc_vcd_value")
        citation["Authors"] = data_field.text
    elif data_category.text == "Publication date":
        data_field = data.find_element_by_class_name("gsc_vcd_value")
        citation["Date"] = data_field.text
    elif data_category.text == "Publisher":
        data_field = data.find_element_by_class_name("gsc_vcd_value")
        citation["Publisher"] = data_field.text
    elif data_category.text == "Description":
        data_field = data.find_element_by_class_name("gsc_vcd_value")
        citation["Description"] = data_field.text
    elif data_category.text == "Journal":
        data_field = data.find_element_by_class_name("gsc_vcd_value")
        citation["Journal"] = data_field.text
    elif data_category.text == "Issue":
        data_field = data.find_element_by_class_name("gsc_vcd_value")
        citation["Issue"] = data_field.text
    elif data_category.text == "Volume":
        data_field = data.find_element_by_class_name("gsc_vcd_value")
        citation["Volume"] = data_field.text
    elif data_category.text == "Pages":
        data_field = data.find_element_by_class_name("gsc_vcd_value")
        citation["Pages"] = data_field.text
        
    counter += 1

print(counter)
# citation_table_data = [data.text for data in citation_table_data_objects]
# for data in citation_table_data:
#     print(data)

# if citation_table.find_elements_by_xpath('//div/div')
pub_title = driver.find_element_by_class_name('gsc_vcd_title_link')
citation["Title"] = pub_title.text


# pub_date = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[2]/div[2]')
# pub_date = pub_date.text
# pub_journal = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[3]/div[2]')
# pub_journal = pub_journal.text
# pub_vol = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[4]/div[2]')
# pub_vol = pub_vol.text
# pub_iss = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[5]/div[2]')
# pub_iss = pub_iss.text
# pub_pgs = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[6]/div[2]')
# pub_pgs = pub_pgs.text
# pub_publisher = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[7]/div[2]')
# pub_publisher = pub_publisher.text
# pub_abstr = driver.find_element_by_xpath('//*[@id="gsc_vcd_descr"]/div/div')
# pub_abstr = pub_abstr.text 
# print(pub_title)
# print(pub_date)
# print(pub_journal)
# print(pub_vol)
# print(pub_iss)
# print(pub_pgs)
# print(pub_publisher)
# print(pub_abstr)

# Retrieve all the urls of the first page
#research_a_tags = driver.find_elements_by_xpath("html/body/div[2]/main/section[2]/div/div[4]/div[2]/div[2]/div/div/div/div/div/div[2]/div/a")
#urls = [tag.get_attribute("href") for tag in research_a_tags]
#titles = [tag.text for tag in research_a_tags]
#paper_types = driver.find_elements_by_xpath('html/body/div[2]/main/section[2]/div/div[4]/div[2]/div[2]/div/div/div/div/div/div[3]/div/div/span')
#paper_types = [item.text for item in paper_types]
#num_research_items -= 100

#current_url = driver.current_url

#page_count = 2
#while num_research_items > 0:
    #driver.get(current_url + '/' + str(page_count))
    #temp_tags = driver.find_elements_by_xpath("html/body/div[2]/main/section[2]/div/div[4]/div[2]/div[2]/div/div/div/div/div/div[2]/div/a")
    #for tag in temp_tags:
        #urls.append(tag.get_attribute("href"))
        #titles.append(tag.text)
    #temp_papers = driver.find_elements_by_xpath('html/body/div[2]/main/section[2]/div/div[4]/div[2]/div[2]/div/div/div/div/div/div[3]/div/div/span')
    #for paper in temp_papers:
        #paper_types.append(paper.text)
    #num_research_items -= 100
    #page_count += 1

#while "Full-text available" in paper_types:
    #paper_types.remove("Full-text available")
driver.close()
    

Not in an infinite loop
[<selenium.webdriver.remote.webelement.WebElement (session="f2dcfb87e8e3e6f6cbdc183b00d77dcc", element="0.17961553428453714-2")>, <selenium.webdriver.remote.webelement.WebElement (session="f2dcfb87e8e3e6f6cbdc183b00d77dcc", element="0.17961553428453714-3")>, <selenium.webdriver.remote.webelement.WebElement (session="f2dcfb87e8e3e6f6cbdc183b00d77dcc", element="0.17961553428453714-4")>, <selenium.webdriver.remote.webelement.WebElement (session="f2dcfb87e8e3e6f6cbdc183b00d77dcc", element="0.17961553428453714-5")>, <selenium.webdriver.remote.webelement.WebElement (session="f2dcfb87e8e3e6f6cbdc183b00d77dcc", element="0.17961553428453714-6")>, <selenium.webdriver.remote.webelement.WebElement (session="f2dcfb87e8e3e6f6cbdc183b00d77dcc", element="0.17961553428453714-7")>, <selenium.webdriver.remote.webelement.WebElement (session="f2dcfb87e8e3e6f6cbdc183b00d77dcc", element="0.17961553428453714-8")>, <selenium.webdriver.remote.webelement.WebElement (session="f2dcfb87e8e3e

ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:void(0)" data-href="/citations?view_op=view_citation&amp;hl=en&amp;user=a5_Ga0cAAAAJ&amp;citation_for_view=a5_Ga0cAAAAJ:u-x6o8ySG0sC" class="gsc_a_at">...</a> is not clickable at point (249, 81). Other element would receive the click: <div id="gs_md_cita-d-bdy" class="gs_md_bdy" style="max-height: 744px;">...</div>
  (Session info: chrome=74.0.3729.131)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.17134 x86_64)


In [55]:
print(citation['Title'])
print(citation['Authors'])
print(citation['Date'])
print(citation['Journal'])
print(citation['Volume'])
print(citation['Issue'])
print(citation['Pages'])
print(citation['Publisher'])
print(citation['Description'])

Tumor-targeted Salmonella as a novel anticancer vector
John M Pawelek, K Brooks Low, David Bermudes
1997/10/15
Cancer research
57
20
4537-4544
American Association for Cancer Research
There has been little investigation of bacteria as gene delivery vectors. Here, we demonstrate that genetically engineered Salmonella have many of the desirable properties of a delivery vector, including targeting of multiple tumors from a distant inoculation site, selective replication within tumors, tumor retardation, and the ability to express effector genes, such as the herpes simplex virus thymidine kinase (HSV TK). When wild-type Salmonella were introduced into melanoma-bearing mice, the bacteria were found within the tumor at levels exceeding 109 per g, although as pathogens, they caused the death of the mice. However, when attenuated, hyperinvasive auxotrophic mutants were used, the tumor-targeting and amplification phenomena were retained, whereas their pathogenicity was limited. With such attenu

In [53]:
count = 0
#for url in urls:
driver = webdriver.Chrome(options=options)
#time.sleep(randint(5,10))

driver.get(urls[0])

# At this point we already have name, title, type, url
# Need citation, collabs, date, DOI, publisher, edition, series, volume, issue, ISSN, ISBN, handle, book title, 

if paper_types[0] == 'Article':
    # this should get the DOI and Abstract in one go
    
    publication_details = driver.find_elements_by_class_name('publication-details__section')
    print(publication_details)
    
    data = publication_details[0].find_elements_by_xpath('//div/div[2]')
    
    print(data)
    
    #meta = driver.find_elements_by_xpath('html/body/div[2]/main/section/section/div/div[2]')
    #print(meta)
    #doi = meta[0].text.split("")
    #doi = doi[1]
    #abstract = meta[1].text
    # then we'll get the collaborators
    
    
    # Attempt to get the date
    #date = driver.find_element_by_xpath('html/body/div[2]/main/section/section/div/div/span[2]')
    
    #download the citatation
    
    # take it apart, perhaps improve it
    
    # build a pandas row
    
    # add it to the main dataframe
    # end loop
    
#print(doi)
#print(abstract)

    
#elif paper_type[0] == "Conference Paper"





driver.close()
    #count +=1

NameError: name 'urls' is not defined

In [99]:
counter = 0
for url in urls:
    print(counter, ":", url)
    counter +=1

0 : https://www.researchgate.net/publication/315416439_Composite_mechanics_of_the_multilayer_structure_of_the_annulus_fibrosus
1 : https://www.researchgate.net/publication/313626183_Modeling_and_Analysis_of_Heart_Left_Ventricle_in_Diastole_Using_Finite_Element
2 : https://www.researchgate.net/publication/285611966_Machine_design_Redesigned
3 : https://www.researchgate.net/publication/261035675_Endoprosthetic_reconstruction_after_resection_of_musculoskeletal
4 : https://www.researchgate.net/publication/40768832_Cemented_Distal_Femoral_Endoprostheses_for_Musculoskeletal_Tumor_Improved_Survival_of_Modular_versus_Custom_Implants
5 : https://www.researchgate.net/publication/44571522_How_Long_Do_Endoprosthetic_Reconstructions_for_Proximal_Femoral_Tumors_Last
6 : https://www.researchgate.net/publication/44624077_Cemented_Endoprosthetic_Reconstruction_of_the_Proximal_Tibia_How_Long_Do_They_Last
7 : https://www.researchgate.net/publication/51381639_Hysteretic_pinching_of_human_secondary_osteons

Note that ResearchGate only loads 100 citations per page, if number of citations % 100 = 0 we go to second page

In [ ]:
gs_bpf_wrapper = driver.find_element_by_id("gsc_bpf")
